In [1]:
import pandas as pd
import os
import requests
FSbrukar = os.environ["FSbrukar"]
FSpassord = os.environ["FSpassord"]

parametreCanvas = {'per_page': '100'}
hodeCanvas = {'Authorization': 'Bearer ' + os.environ["tokenCanvas"]}

data = pd.read_csv('sider.csv', sep=',')

In [2]:
def query_FS_graphql(query, variable):
    hode = {
        'Accept': 'application/json;version=1',
        "Feature-Flags": "beta, experimental"
    }
    GraphQLurl = "https://api.fellesstudentsystem.no/graphql/"
    svar = requests.post(
        GraphQLurl, 
        json = {
            'query': query,
            'variables': variable
        },
        headers=hode,
        auth=(FSbrukar, FSpassord))
    if 200 <= svar.status_code < 300:
        return svar.json()
    else:
        return {}

In [3]:
query = """
    query MyQuery($emnekode: [String!]) {
    emner(
        filter: {eierInstitusjonsnummer: "203", emnekoder: $emnekode, ikkeUtloptITermin: {arstall: 2025, terminbetegnelse: VAR}}
    )  {
    nodes {
      personroller(filter: {erAktiv: true}) {
        fsRolle {
          navn {
            publiseringsnavn
          }
          erAktiv
        }
        fagperson {
          navn {
            etternavn
            fornavn
          }
          feideBruker
        }
        emne {
          navnAlleSprak {
            nn
          }
        }
      }
    }
  }
}
    """

In [4]:
data['emne'] = data['index'].str.split('/', expand=True)[2]

In [5]:
for s in data['emne'].value_counts().index[0:10]:
    canvasurl = f"https://hvl.instructure.com/api/v1/courses/{s}"
    responsCanvas = requests.get(canvasurl, headers=hodeCanvas, params=parametreCanvas)
    if 200 <= responsCanvas.status_code < 300:
        dataCanvas = responsCanvas.json()
        if dataCanvas['sis_course_id'][0:1] == "U":
            emnekode = dataCanvas['sis_course_id'].split('_')[2]
            variable = {"emnekode": emnekode}
            svar = query_FS_graphql(query, variable)
            namn = svar['data']['emner']['nodes'][0]['personroller'][0]['fagperson']['navn']['fornavn'] + ' ' + svar['data']['emner']['nodes'][0]['personroller'][0]['fagperson']['navn']['etternavn']
            brukarnamn = svar['data']['emner']['nodes'][0]['personroller'][0]['fagperson']['feideBruker']
            emnenamn = svar['data']['emner']['nodes'][0]['personroller'][0]['emne']['navnAlleSprak']['nn']
            print(f"{emnekode} {emnenamn}: {namn}, {brukarnamn}")

BIO207 Ekstern praksis 1: Anita Ryningen, ary@hvl.no
VPD200 Vernepleierens helsefaglege kompetanse: Agnes Merete Fotland, agnesf@hvl.no
ENG701 Engelsk lingvistikk 2: Andre Storto, ansto@hvl.no
FYS130 Grunnleggjande fysiologi : Maria Nordheim Alme, mana@hvl.no
GE483 Climate Change: Mette Kusk Gillespie, metteri@hvl.no
REAL111 Fysikk : Kyrre Skjerdal, kskj@hvl.no
VPL130 Miljøterapeutisk arbeid : Astrid Steffensen, asst@hvl.no
MAMET1HE Vitskapsteori, etikk og metode: Anne-Siri Fismen, asfi@hvl.no
ELE142 Elektrofagleg basis 2: Bjarte Pollen, bjartepo@hvl.no


In [6]:
data

,index,frekvens,emne
0,/courses/29065/pages/world-climate-simulation,62,29065
1,/courses/30211/pages/mal-for-oppgaver-i-praksi...,50,30211
2,/courses/28875/pages/torsdag-27-dot-2-f12-sirk...,48,28875
3,/courses/28114/pages/uke-9-kap-4-dot-1-4-dot-3...,48,28114
4,/courses/28959/pages/farmakologi-oppgaver-2?mo...,40,28959
...,...,...,...
2380,/courses/28709/pages/veke-4-og-5-teorem-r-and-...,1,28709
2381,/courses/28525/pages/laeringsmal-og-oppgaver-f...,1,28525
2382,/courses/30100/pages/aktivitet-sovn-og-kvile-2...,1,30100
2383,/courses/29872/pages/vekselvirkningsprosesser-...,1,29872


In [12]:
data[data['emne']=="28490"].count()['frekvens']

93